In [1]:
import random
import numpy as np

import pytest
from datasetsforecast.m4 import M4, M4Info

from mlforecast.lag_transforms import SeasonalRollingMean
from mlforecast.lgb_cv import LightGBMCV
from mlforecast.target_transforms import Differences
from mlforecast.utils import generate_daily_series, generate_prices_for_series


# @pytest.fixture(scope="module")
def m4_data():
    group = "Hourly"
    M4.async_download("data", group=group)
    df, *_ = M4.load(directory="data", group=group)
    df["ds"] = df["ds"].astype("int")
    ids = df["unique_id"].unique()
    random.seed(0)
    sample_ids = random.choices(ids, k=4)
    sample_df = df[df["unique_id"].isin(sample_ids)]
    info = M4Info[group]
    horizon = info.horizon
    valid = sample_df.groupby("unique_id").tail(horizon)
    train = sample_df.drop(valid.index)
    return train, valid, horizon


def evaluate_on_valid(preds, valid):
    preds = preds.copy()
    preds["final_prediction"] = preds.drop(columns=["unique_id", "ds"]).mean(1)
    merged = preds.merge(valid, on=["unique_id", "ds"])
    merged["abs_err"] = abs(merged["final_prediction"] - merged["y"]) / merged["y"]
    return merged.groupby("unique_id")["abs_err"].mean().mean()


# @pytest.mark.parametrize("use_weight_col", [True, False])
def test_lightgbm_cv_pipeline(m4_data, use_weight_col):
    train, valid, horizon = m4_data
    if use_weight_col:
        train["weight_col"] = 1
    static_fit_config = dict(
        n_windows=2,
        h=horizon,
        params={"verbose": -1},
        compute_cv_preds=True,
    )
    cv = LightGBMCV(
        freq=1,
        lags=[24 * (i + 1) for i in range(7)],
    )
    #hist = cv.fit(train, **static_fit_config, weight_col='weight_col' if use_weight_col else None)
    hist = cv.fit(train, **static_fit_config, weight_col='weight_col')
    preds = cv.predict(horizon)
    eval1 = evaluate_on_valid(preds, valid)

    cv2 = LightGBMCV(
        freq=1,
        target_transforms=[Differences([24 * 7])],
        lags=[24 * (i + 1) for i in range(7)],
    )
    #hist2 = cv2.fit(train, **static_fit_config, weight_col='weight_col' if use_weight_col else None)
    hist2 = cv2.fit(train, **static_fit_config, weight_col='weight_col')
    print(hist2[-1][1])
    
    print(hist[-1][1])
    
    assert hist2[-1][1] < hist[-1][1]
    preds2 = cv2.predict(horizon)
    eval2 = evaluate_on_valid(preds2, valid)
    assert eval2 < eval1

    cv3 = LightGBMCV(
        freq=1,
        target_transforms=[Differences([24 * 7])],
        lags=[24 * (i + 1) for i in range(7)],
        lag_transforms={48: [SeasonalRollingMean(season_length=24, window_size=7)],
        }
    )
    hist3 = cv3.fit(train, **static_fit_config, weight_col='weight_col' if use_weight_col else None)
    assert hist3[-1][1] < hist2[-1][1]
    # preds3 = cv3.predict(horizon)
    # eval3 = evaluate_on_valid(preds3, valid)

    assert cv.find_best_iter([(0, 1), (1, 0.5)], 1) == 1
    assert cv.find_best_iter([(0, 1), (1, 0.5), (2, 0.6)], 1) == 1
    assert cv.find_best_iter([(0, 1), (1, 0.5), (2, 0.6), (3, 0.4)], 2) == 3

    cv4 = LightGBMCV(
        freq=1,
        lags=[24 * (i + 1) for i in range(7)],
    )
    cv4.setup(
        train,
        n_windows=2,
        h=horizon,
        params={"verbose": -1},
    )
    score = cv4.partial_fit(10)
    assert np.isclose(hist[0][1], score, atol=1e-7)
    score2 = cv4.partial_fit(20)
    assert np.isclose(hist[2][1], score2, atol=1e-7)


test_lightgbm_cv_pipeline(m4_data(), use_weight_col=True)


/var/folders/nk/kvcs64mn4nbfqfw1ff_czjn80000gn/T/ipykernel_39999/2339772901.py:16: RuntimeWarning: coroutine 'M4.async_download' was never awaited
  M4.async_download("data", group=group)


[10] mape: 0.590690
[20] mape: 0.251093
[30] mape: 0.143643
[40] mape: 0.109723
[50] mape: 0.102099
[60] mape: 0.099448
[70] mape: 0.098349
[80] mape: 0.098006
[90] mape: 0.098718
Early stopping at round 90
Using best iteration: 80
[10] mape: 0.089024
[20] mape: 0.090683
[30] mape: 0.092316
Early stopping at round 30
Using best iteration: 10
0.08902401935024312
0.09800559741019788
[10] mape: 0.086724
[20] mape: 0.088466
[30] mape: 0.090536
Early stopping at round 30
Using best iteration: 10


In [29]:
use_weight_col = False

train, valid, horizon = m4_data()
if use_weight_col:
    train["weight_col"] = 1
static_fit_config = dict(
    n_windows=2,
    h=horizon,
    params={"verbose": -1},
    compute_cv_preds=True,
)
cv = LightGBMCV(
    freq=1,
    lags=[24 * (i + 1) for i in range(7)],
)
#hist = cv.fit(train, **static_fit_config, weight_col='weight_col' if use_weight_col else None)
hist = cv.fit(train, **static_fit_config, 
              #weight_col='weight_col', 
              metric='rmse')
preds = cv.predict(horizon)
eval1 = evaluate_on_valid(preds, valid)

cv2 = LightGBMCV(
    freq=1,
    target_transforms=[Differences([24 * 7])],
    lags=[24 * (i + 1) for i in range(7)],
)
#hist2 = cv2.fit(train, **static_fit_config, weight_col='weight_col' if use_weight_col else None)
hist2 = cv2.fit(train, **static_fit_config, 
                #weight_col='weight_col',
                metric='rmse')

assert hist2[-1][1] > hist[-1][1]
preds2 = cv2.predict(horizon)
eval2 = evaluate_on_valid(preds2, valid)
assert eval2 < eval1

cv3 = LightGBMCV(
        freq=1,
        target_transforms=[Differences([24 * 7])],
        lags=[24 * (i + 1) for i in range(7)],
        lag_transforms={48: [SeasonalRollingMean(season_length=24, window_size=7)],
        }
    )
hist3 = cv3.fit(train, **static_fit_config, 
                # weight_col='weight_col' if use_weight_col else None
                metric='rmse'
                )
assert hist3[-1][1] < hist2[-1][1]
# preds3 = cv3.predict(horizon)
# eval3 = evaluate_on_valid(preds3, valid)

assert cv.find_best_iter([(0, 1), (1, 0.5)], 1) == 1
assert cv.find_best_iter([(0, 1), (1, 0.5), (2, 0.6)], 1) == 1
assert cv.find_best_iter([(0, 1), (1, 0.5), (2, 0.6), (3, 0.4)], 2) == 3


cv4 = LightGBMCV(
        freq=1,
        lags=[24 * (i + 1) for i in range(7)],
    )
cv4.setup(
    train,
    n_windows=2,
    h=horizon,
    params={"verbose": -1},
    metric='rmse'
)
score = cv4.partial_fit(10)
assert np.isclose(hist[0][1], score, atol=1e-7)
score2 = cv4.partial_fit(20)
assert np.isclose(hist[2][1], score2, atol=1e-7)

/var/folders/nk/kvcs64mn4nbfqfw1ff_czjn80000gn/T/ipykernel_39999/2339772901.py:16: RuntimeWarning: coroutine 'M4.async_download' was never awaited
  M4.async_download("data", group=group)


[10] rmse: 25.068319
[20] rmse: 13.732348
[30] rmse: 12.174253
[40] rmse: 12.416480
[50] rmse: 12.557813
Early stopping at round 50
Using best iteration: 30
[10] rmse: 14.202833
[20] rmse: 13.946873
[30] rmse: 14.401254
Early stopping at round 30
Using best iteration: 20
[10] rmse: 14.161068
[20] rmse: 13.513297
[30] rmse: 13.742724
[40] rmse: 13.912033
Early stopping at round 40
Using best iteration: 20


In [ ]:
import pytest
import numpy as np

@pytest.mark.parametrize("use_weight_col", [True, False])
@pytest.mark.parametrize("metric", ["rmse", "mape"])   # add whatever metrics you support
def test_lightgbm_cv_pipeline(m4_data, use_weight_col, metric):
    train, valid, horizon = m4_data
    if use_weight_col:
        train["weight_col"] = 1

    static_fit_config = dict(
        n_windows=2,
        h=horizon,
        params={"verbose": -1},
        compute_cv_preds=True,
        metric=metric,  # <-- pass through
    )

    cv = LightGBMCV(freq=1, lags=[24 * (i + 1) for i in range(7)])
    hist = cv.fit(train, **static_fit_config, weight_col='weight_col' if use_weight_col else None)
    preds = cv.predict(horizon)
    eval1 = evaluate_on_valid(preds, valid)

    cv2 = LightGBMCV(
        freq=1,
        target_transforms=[Differences([24 * 7])],
        lags=[24 * (i + 1) for i in range(7)],
    )
    hist2 = cv2.fit(train, **static_fit_config, weight_col='weight_col' if use_weight_col else None)

    # These "improves over" assertions may not hold for every metric.
    # Keep them only for metrics where it's stable.
    if metric == "rmse":
        assert hist2[-1][1] < hist[-1][1]

    preds2 = cv2.predict(horizon)
    eval2 = evaluate_on_valid(preds2, valid)
    if metric == "rmse":
        assert eval2 < eval1

    cv3 = LightGBMCV(
        freq=1,
        target_transforms=[Differences([24 * 7])],
        lags=[24 * (i + 1) for i in range(7)],
        lag_transforms={48: [SeasonalRollingMean(season_length=24, window_size=7)]},
    )
    hist3 = cv3.fit(train, **static_fit_config, weight_col='weight_col' if use_weight_col else None)
    if metric == "rmse":
        assert hist3[-1][1] < hist2[-1][1]

    assert cv.find_best_iter([(0, 1), (1, 0.5)], 1) == 1
    assert cv.find_best_iter([(0, 1), (1, 0.5), (2, 0.6)], 1) == 1
    assert cv.find_best_iter([(0, 1), (1, 0.5), (2, 0.6), (3, 0.4)], 2) == 3

    cv4 = LightGBMCV(freq=1, lags=[24 * (i + 1) for i in range(7)])
    cv4.setup(train, n_windows=2, h=horizon, params={"verbose": -1}, metric=metric)
    score = cv4.partial_fit(10)
    assert np.isclose(hist[0][1], score, atol=1e-7)
    score2 = cv4.partial_fit(20)
    assert np.isclose(hist[2][1], score2, atol=1e-7)


### New backtest splits

In [1]:
import os
import tempfile

import lightgbm as lgb
import optuna
import pandas as pd
from datasetsforecast.m4 import M4, M4Evaluation, M4Info
from sklearn.linear_model import Ridge
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from utilsforecast.plotting import plot_series

from mlforecast import MLForecast
from mlforecast.auto import (
    AutoLightGBM,
    AutoMLForecast,
    AutoModel,
    AutoRidge,
    ridge_space,
)
from mlforecast.lag_transforms import ExponentiallyWeightedMean, RollingMean

/Users/janrathfelder/miniconda3/envs/mlforecast-dev/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_data(group, horizon):
    df, *_ = M4.load(directory='data', group=group)
    df['ds'] = df['ds'].astype('int')
    df['unique_id'] = df['unique_id'].astype('category')
    return df.groupby('unique_id').head(-horizon).copy()

group = 'Hourly'
horizon = M4Info[group].horizon
train = get_data(group, horizon)

/var/folders/nk/kvcs64mn4nbfqfw1ff_czjn80000gn/T/ipykernel_46590/3410956497.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return df.groupby('unique_id').head(-horizon).copy()


In [3]:
optuna.logging.set_verbosity(optuna.logging.ERROR)
auto_mlf = AutoMLForecast(
    models={'lgb': AutoLightGBM(), 'ridge': AutoRidge()},
    freq=1,
    season_length=24,
    reuse_cv_splits=True
)
auto_mlf.fit(
    train,
    n_windows=10,
    h=horizon,
    num_samples=20,  # number of trials to run
)

AutoMLForecast(models={'lgb': AutoModel(model=LGBMRegressor), 'ridge': AutoModel(model=Ridge)})

In [29]:
series.shape

(14470005, 3)

In [60]:
from utilsforecast.data import generate_series

series = generate_series(n_series=20000, freq='D', min_length=722, max_length=725, equal_ends=True)


# Setting up model
def my_lgb_config(trial: optuna.Trial):
    return {
        'learning_rate': 0.05,
        'verbosity': -1,
        'num_leaves': trial.suggest_int('num_leaves', 127, 128, log=True),
    }
    
def my_fit_config(trial: optuna.Trial):
    return {'dropna':False
    }
    
def my_init_config(trial: optuna.Trial):
    return {
        'lags': [24 * i for i in range(1, 3)],  # this won't be tuned
        'lag_transforms' : {
    1: [ExponentiallyWeightedMean(alpha=0.3),
        RollingMean(window_size=7, min_samples=1),
         RollingMean(window_size=21, min_samples=1),
          RollingMean(window_size=28, min_samples=1),
           RollingMean(window_size=50, min_samples=1),
        ],
}}

my_lgb = AutoModel(
    model=lgb.LGBMRegressor(),
    config=my_lgb_config,
)

import time

# -------------------------------
# Fitting model WITHOUT reusing CV splits
# -------------------------------
t0 = time.perf_counter()

auto_mlf_no_reuse = AutoMLForecast(
    models={"my_lgb": my_lgb},
    freq="D",
    season_length=24,
    fit_config=my_fit_config,
    init_config=my_init_config,
    reuse_cv_splits=False,
).fit(
    series,
    n_windows=15,
    h=horizon,
    num_samples=10,
)

t_no_reuse = time.perf_counter() - t0
print(f"Fit time without CV split reuse: {t_no_reuse:.2f} seconds")

# -------------------------------
# Fitting model WITH reusing CV splits
# -------------------------------
t0 = time.perf_counter()

auto_mlf_reuse = AutoMLForecast(
    models={"my_lgb": my_lgb},
    freq="D",
    season_length=24,
    fit_config=my_fit_config,
    init_config=my_init_config,
    reuse_cv_splits=True,   # <-- important
).fit(
    series,
    n_windows=15,
    h=horizon,
    num_samples=10,
)

t_reuse = time.perf_counter() - t0
print(f"Fit time with CV split reuse: {t_reuse:.2f} seconds")

# -------------------------------
# Comparison
# -------------------------------
speedup = t_no_reuse / t_reuse
delta = t_no_reuse - t_reuse

print(f"Speedup: {speedup:.2f}×")
print(f"Time saved: {delta:.2f} seconds ({delta / t_no_reuse * 100:.1f}%)")


/Users/janrathfelder/Documents/data_science/GitHub/mlforecast/mlforecast/auto.py:267: UserWarning: `season_length` is not used when `init_config` is provided.
  warnings.warn("`season_length` is not used when `init_config` is provided.")
/Users/janrathfelder/Documents/data_science/GitHub/mlforecast/mlforecast/core.py:622: UserWarning: Found null values in lag24, lag48.
  warnings.warn(f'Found null values in {", ".join(cols_with_nulls)}.')
/Users/janrathfelder/Documents/data_science/GitHub/mlforecast/mlforecast/core.py:622: UserWarning: Found null values in lag24, lag48.
  warnings.warn(f'Found null values in {", ".join(cols_with_nulls)}.')
/Users/janrathfelder/Documents/data_science/GitHub/mlforecast/mlforecast/core.py:622: UserWarning: Found null values in lag24, lag48.
  warnings.warn(f'Found null values in {", ".join(cols_with_nulls)}.')
/Users/janrathfelder/Documents/data_science/GitHub/mlforecast/mlforecast/core.py:622: UserWarning: Found null values in lag24, lag48.
  warnings.wa

Fit time without CV split reuse: 521.28 seconds


/Users/janrathfelder/Documents/data_science/GitHub/mlforecast/mlforecast/auto.py:267: UserWarning: `season_length` is not used when `init_config` is provided.
  warnings.warn("`season_length` is not used when `init_config` is provided.")
/Users/janrathfelder/Documents/data_science/GitHub/mlforecast/mlforecast/core.py:622: UserWarning: Found null values in lag24, lag48.
  warnings.warn(f'Found null values in {", ".join(cols_with_nulls)}.')
/Users/janrathfelder/Documents/data_science/GitHub/mlforecast/mlforecast/core.py:622: UserWarning: Found null values in lag24, lag48.
  warnings.warn(f'Found null values in {", ".join(cols_with_nulls)}.')
/Users/janrathfelder/Documents/data_science/GitHub/mlforecast/mlforecast/core.py:622: UserWarning: Found null values in lag24, lag48.
  warnings.warn(f'Found null values in {", ".join(cols_with_nulls)}.')
/Users/janrathfelder/Documents/data_science/GitHub/mlforecast/mlforecast/core.py:622: UserWarning: Found null values in lag24, lag48.
  warnings.wa

Fit time with CV split reuse: 457.77 seconds
Speedup: 1.14×
Time saved: 63.51 seconds (12.2%)


In [16]:
train.shape

(353500, 3)

In [36]:
def my_lgb_config(trial: optuna.Trial):
    return {
        'learning_rate': 0.05,
        'verbosity': -1,
        'num_leaves': trial.suggest_int('num_leaves', 127, 128, log=True),
    }
    
def my_fit_config(trial: optuna.Trial):
    return {'dropna':False
    }
    
def my_init_config(trial: optuna.Trial):
    return {
        'lags': [24 * i for i in range(1, 3)],  # this won't be tuned
        'lag_transforms' : {
    1: [ExponentiallyWeightedMean(alpha=0.3),
        RollingMean(window_size=7, min_samples=1),
         RollingMean(window_size=21, min_samples=1),
          RollingMean(window_size=28, min_samples=1),
           RollingMean(window_size=50, min_samples=1),
        ],
}}
    
my_lgb = AutoModel(
    model=lgb.LGBMRegressor(),
    config=my_lgb_config,
   
)
auto_mlf = AutoMLForecast(
    models={'my_lgb': my_lgb},
    freq='D',
    season_length=24,
    fit_config=my_fit_config,
    init_config=my_init_config,
    reuse_cv_splits=True
).fit(
    series,
    n_windows=15,
    h=horizon,
    num_samples=10,
)

/Users/janrathfelder/Documents/data_science/GitHub/mlforecast/mlforecast/auto.py:267: UserWarning: `season_length` is not used when `init_config` is provided.
  warnings.warn("`season_length` is not used when `init_config` is provided.")
/Users/janrathfelder/Documents/data_science/GitHub/mlforecast/mlforecast/core.py:622: UserWarning: Found null values in lag24, lag48.
  warnings.warn(f'Found null values in {", ".join(cols_with_nulls)}.')
/Users/janrathfelder/Documents/data_science/GitHub/mlforecast/mlforecast/core.py:622: UserWarning: Found null values in lag24, lag48.
  warnings.warn(f'Found null values in {", ".join(cols_with_nulls)}.')
/Users/janrathfelder/Documents/data_science/GitHub/mlforecast/mlforecast/core.py:622: UserWarning: Found null values in lag24, lag48.
  warnings.warn(f'Found null values in {", ".join(cols_with_nulls)}.')
/Users/janrathfelder/Documents/data_science/GitHub/mlforecast/mlforecast/core.py:622: UserWarning: Found null values in lag24, lag48.
  warnings.wa

In [37]:
7.32 / 8.47 - 1

-0.13577331759149946

In [38]:
452 / 527 - 1

-0.14231499051233398

In [58]:
import numpy as np
import pandas as pd
import pytest
from sklearn.linear_model import Ridge

from mlforecast.auto import AutoMLForecast, AutoModel

from datasetsforecast.m4 import M4, M4Info


def weekly_data():
    group = "Weekly"
    M4.async_download("data", group=group)
    df, *_ = M4.load(directory="data", group=group)
    df["ds"] = df["ds"].astype("int")
    horizon = M4Info[group].horizon
    valid = df.groupby("unique_id").tail(horizon).copy()
    train = df.drop(valid.index).reset_index(drop=True)
    train["unique_id"] = train["unique_id"].astype("category")
    valid["unique_id"] = valid["unique_id"].astype(train["unique_id"].dtype)
    return train, valid, M4Info[group]


def test_reuse_cv_splits_same_predictions():
    train, valid, info = weekly_data()
    h = info.horizon
    n_windows = 2
    num_samples = 5 
    
    # deterministic AutoModel: config doesn't depend on trial
    def ridge_config(trial):  # noqa: ARG001
        return {"alpha": 1.0, "fit_intercept": True, "solver": "svd"}

    def fit_config(trial):  # noqa: ARG001
        return {"dropna": True}

    def init_config(trial):  # noqa: ARG001
        return {
            "lags": [1, 2, 3],
            # no randomness, keep it simple
        }

    ridge_auto = AutoModel(model=Ridge(), config=ridge_config)

    common_kwargs = dict(
        models={"ridge": ridge_auto},
        freq=1,
        fit_config=fit_config,
        init_config=init_config,
    )

    # Run without reuse
    automl_a = AutoMLForecast(**common_kwargs, reuse_cv_splits=False).fit(
        train,
        n_windows=n_windows,
        h=h,
        num_samples=num_samples,
    )

    # Run with reuse
    automl_b = AutoMLForecast(**common_kwargs, reuse_cv_splits=True).fit(
        train,
        n_windows=n_windows,
        h=h,
        num_samples=num_samples,
    )

    preds_a = automl_a.predict(h=h)
    preds_b = automl_b.predict(h=h)

    # Align and compare
    #preds_a = preds_a.sort_values(["unique_id", "ds"]).reset_index(drop=True)
    #preds_b = preds_b.sort_values(["unique_id", "ds"]).reset_index(drop=True)

    # The prediction column name is the model key ("ridge")
    
    print(preds_a["ridge"].to_numpy())
          
    print(preds_b["ridge"].to_numpy())
    
    assert preds_a.columns.tolist() == preds_b.columns.tolist()
    assert (preds_a["ridge"].to_numpy() == preds_b["ridge"].to_numpy()).all()



In [59]:
test_reuse_cv_splits_same_predictions()

/var/folders/nk/kvcs64mn4nbfqfw1ff_czjn80000gn/T/ipykernel_46590/3797830855.py:13: RuntimeWarning: coroutine 'M4.async_download' was never awaited
  M4.async_download("data", group=group)


[35483.33419905 35485.00109076 35249.23386033 ... 14553.00311463
 14504.48644501 14456.19849454]
[35483.33419905 35485.00109076 35249.23386033 ... 14553.00311463
 14504.48644501 14456.19849454]


In [3]:
preds

,unique_id,ds,Booster0,Booster1
0,H196,961,17.167009,17.374538
1,H196,962,17.063952,17.223050
2,H196,963,16.526860,16.691864
3,H196,964,16.526860,16.248888
4,H196,965,16.103180,15.976743
...,...,...,...,...
187,H413,1004,72.331252,69.121673
188,H413,1005,70.079689,67.190393
189,H413,1006,63.003196,63.165443
190,H413,1007,48.460301,51.193604


In [7]:
train.head()

,unique_id,ds,y,weight_col
86796,H196,1,11.8,1
86797,H196,2,11.4,1
86798,H196,3,11.1,1
86799,H196,4,10.8,1
86800,H196,5,10.6,1


In [ ]:
def test_lightgbmcv_callback():
    def before_predict_callback(df):
        assert not df["price"].isnull().any()
        return df

    dynamic_series = generate_daily_series(
        100, equal_ends=True, n_static_features=2, static_as_categorical=False
    )
    dynamic_series = dynamic_series.rename(columns={"static_1": "product_id"})
    prices_catalog = generate_prices_for_series(dynamic_series)
    series_with_prices = dynamic_series.merge(prices_catalog, how="left")
    cv = LightGBMCV(freq="D", lags=[24])
    _ = cv.fit(
        series_with_prices,
        n_windows=2,
        h=5,
        params={"verbosity": -1},
        static_features=["static_0", "product_id"],
        verbose_eval=False,
        before_predict_callback=before_predict_callback,
    )


def test_lightgbmcv_custom_metric(m4_data):
    train, _, horizon = m4_data

    def weighted_mape(y_true, y_pred, ids, dates):
        abs_pct_err = abs(y_true - y_pred) / abs(y_true)
        mape_by_serie = abs_pct_err.groupby(ids).mean()
        totals_per_serie = y_pred.groupby(ids).sum()
        series_weights = totals_per_serie / totals_per_serie.sum()
        return (mape_by_serie * series_weights).sum()

    _ = LightGBMCV(
        freq=1,
        lags=[24 * (i + 1) for i in range(7)],
    ).fit(
        train,
        n_windows=2,
        h=horizon,
        params={"verbose": -1},
        metric=weighted_mape,
    )


In [1]:
from mlforecast.utils import generate_daily_series

df = generate_daily_series(n_series=20000,  min_length=700,
    max_length=800,
    equal_ends=True,)

In [13]:
from sklearn.base import BaseEstimator
import pandas as pd
from mlforecast import MLForecast

class Naive(BaseEstimator):
    def fit(self, X, y):
        return self

    def predict(self, X):
        return X['lag1']

df_train = df[df.ds < df.ds.max() - pd.Timedelta(days=120)]
update_df = df[df.ds > df_train.ds.max()]

print(df_train.shape)
print(update_df.shape)

fcst = MLForecast(
    models=[Naive()],
    freq='D',
    lags=[1, 2, 3],
)
fcst.fit(df_train)

(12581539, 3)
(2420000, 3)


MLForecast(models=[Naive], freq=D, lag_features=['lag1', 'lag2', 'lag3'], date_features=[], num_threads=1)

In [25]:
df_train.ds.max()

Timestamp('2001-11-09 00:00:00')

In [26]:
update_df.ds.min()

Timestamp('2001-11-09 00:00:00')

In [14]:
update_df = update_df.drop(update_df.index[3])

In [ ]:
fcst.update(update_df, validate_input=True)
preds = fcst.predict(1)

ValueError: Found gaps or duplicate timestamps in the update for: ['id_00000'].

In [34]:
preds.head()

,unique_id,ds,Naive
0,id_00000,2002-03-11,1.082564
1,id_00001,2002-03-11,6.342886
2,id_00002,2002-03-11,0.082227
3,id_00003,2002-03-11,4.152536
4,id_00004,2002-03-11,1.046495


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datasetsforecast.m4 import M4
from sklearn.linear_model import Ridge

from mlforecast import MLForecast
from mlforecast.target_transforms import AutoDifferences

data_path = 'data'
await M4.async_download(data_path, group='Hourly')
df, *_ = M4.load(data_path, 'Hourly')
df['ds'] = df['ds'].astype('int32')
serie = df[df['unique_id'].eq('H196')]

mlfcst = MLForecast(
    models={'ridge': Ridge()},
    freq=1,
    lags=[1, 2, 3, 24],
    target_transforms=[AutoDifferences(max_diffs=1)],
)

without_trend_and_seasonality = mlfcst.preprocess(serie)

mlfcst.fit(
    without_trend_and_seasonality.dropna(),
    id_col='unique_id',
    time_col='ds',
    target_col='y',
    fitted=True,
    dropna=True,
)

h = 24
preds = mlfcst.predict(h=h)
preds = preds[preds['unique_id'] == 'H196']

history = serie[['ds', 'y']].copy()
history_tail = history.tail(200)

plt.figure(figsize=(10, 4))
plt.plot(history_tail['ds'], history_tail['y'], label='history')
plt.plot(preds['ds'], preds['ridge'], label='forecast')
plt.legend()
plt.title('H196 forecast')
plt.show()
